In [1]:
# import required packages
import geopandas as gpd
import pandas as pd
import fiona

In [2]:
#load .gdb connection - NOTE: this points directory to your MSI home folder and Tier 1 storage. If the file is elsewhere you may need to do some additional directory pointing.
gdb_path = "nasispedonsAK.gdb"

In [3]:
#get a list of layers in the .gdb
layers = fiona.listlayers(gdb_path)

In [4]:
# create an object that points to one specific layer - NOTE: here I am pointing to the "pedon" layer, which contains the geometry (lat lon) and many other site level variables
pedon_gdf = gpd.read_file(gdb_path, layer="pedon") 

# inspect the head
print(pedon_gdf.head()) 

# inspect the columns
print(pedon_gdf.columns)

   siteobsiidref        upedonid    pedrecorigin       descname taxonname  \
0        1882228  TNF_NFVEG_4798  USFS NRM pedon  Jennifer A Nu      None   
1        1881710  TNF_NFVEG_4799  USFS NRM pedon  Jennifer A Nu      None   
2        1882881  TNF_NFVEG_4800  USFS NRM pedon  Jennifer A Nu      None   
3        1883259  TNF_NFVEG_4801  USFS NRM pedon  Jennifer A Nu      None   
4        1882958  TNF_NFVEG_4802  USFS NRM pedon  Jennifer A Nu      None   

  localphase taxclname taxonkind pedontype pedonpurpose  ...  sasexposureend  \
0       None      None      None      None         None  ...             NaT   
1       None      None      None      None         None  ...             NaT   
2       None      None      None      None         None  ...             NaT   
3       None      None      None      None         None  ...             NaT   
4       None      None      None      None         None  ...             NaT   

  pedbiidref grpiidref              objwlupdated objuser

In [6]:
# create an object for the horizon table
horizon_gdf = gpd.read_file(gdb_path, layer="phorizon") 

# inspect the head
print(horizon_gdf.head()) 

# inspect the columns
print(horizon_gdf.columns)

   peiidref  seqnum  hzdept  hzdepb  hzthk_l  hzthk_r  hzthk_h obsmethod  \
0   1567867     1.0     NaN     NaN      NaN      NaN      NaN      None   
1   1567869     1.0     0.0     8.0      NaN      NaN      NaN      None   
2   1567869     2.0     8.0     NaN      NaN      NaN      NaN      None   
3   1568367     5.0    10.0   120.0      NaN      NaN      NaN      None   
4   1568367     1.0     0.0     2.0      NaN      NaN      NaN      None   

  hzname  desgndisc  ... horzlatareapct_l horzlatareapct_r  horzlatareapct_h  \
0     Oi        NaN  ...             None             None              None   
1   A/Bw        NaN  ...             None             None              None   
2      C        NaN  ...             None             None              None   
3   None        NaN  ...             None             None              None   
4   None        NaN  ...             None             None              None   

  dspcomplayerid  aashtocl  unifiedcl              recwlupdate

In [9]:
# Step 1: Calculate the thickness of each horizon
horizon_gdf['thickness'] = horizon_gdf['hzdepb'] - horizon_gdf['hzdept']

# Step 2: Filter for horizons with "O" in the hzname
o_horizons = horizon_gdf[horizon_gdf['hzname'].str.contains('O', na=False)]

# Step 3: Group by pedon ID and sum the thickness
total_o_thickness = o_horizons.groupby('peiidref')['thickness'].sum().reset_index()

# Rename the column for clarity
total_o_thickness.rename(columns={'thickness': 'total_O_thickness'}, inplace=True)

# Display the result
print(total_o_thickness)

       peiidref  total_O_thickness
0        107546                6.0
1        107547                2.0
2        107548                0.0
3        107549                3.0
4        107550               27.0
...         ...                ...
23214   1679621               12.0
23215   1679622               39.0
23216   1679623               10.0
23217   1679624               12.0
23218   1679625               60.0

[23219 rows x 2 columns]


#chatgpt ok - now it gets a bit more complicated. There are some pedons where the O horizon overlaps with another horizon (so for example, two horizons might look like this): peiidref, hzdept, hzdepb, hzname; 123456, 0, 10, Oi; 123456, 0, 35, C. In this case, I would like to weight the O thickness by the number of other overlapping horizons. So, for example in the case I gave, the "normal" sum O thickness would be 10cm. However, because there is one overlapping horizon, then it would be 5cm. I'll give a few other examples. in this case: peiidref, hzdept, hzdepb, hzname; 123456, 0, 10, Oi; 123456, 0, 35, C, 123456, 0, 20, Bw. The "normal" sum would be 10cm, but because there are two overlapping horizons, the adjusted sum would be 3.33...cm. Final example: peiidref, hzdept, hzdepb, hzname; 123456, 0, 10, Oi; 123456, 0, 5, Oe; 123456, 5, 10, Bw. In this case, the "normal" thickness would be 5cm. However, the first 5cm are overlapping O's, which would count for 5. The 5-10 depth increment is an O with a competing non-O, so the thickness for that would be 2.5. Therefore, the corrected sum would be 7.5 

To address the overlapping horizons and adjust the O horizon thickness accordingly, we can break this task down into several steps:

Identify Overlaps: For each O horizon, we need to determine how many other horizons overlap with it.
Calculate Adjusted Thickness: Based on the overlaps, we will adjust the thickness of the O horizons.
Aggregate Results: Finally, we will sum the adjusted thicknesses for each unique pedon ID.

In [10]:
import geopandas as gpd
import pandas as pd

# Assuming you already have the horizon_gdf DataFrame loaded

# Step 1: Calculate the thickness of each horizon
horizon_gdf['thickness'] = horizon_gdf['hzdepb'] - horizon_gdf['hzdept']

# Step 2: Filter for horizons with "O" in the hzname
o_horizons = horizon_gdf[horizon_gdf['hzname'].str.contains('O', na=False)]

# Step 3: Initialize a list to hold the adjusted thicknesses
adjusted_thicknesses = []

# Step 4: Loop through each O horizon and calculate adjusted thickness
for index, o_row in o_horizons.iterrows():
    o_start = o_row['hzdept']
    o_end = o_row['hzdepb']
    
    # Get other horizons that overlap with the current O horizon
    overlapping_horizons = horizon_gdf[
        (horizon_gdf['peiidref'] == o_row['peiidref']) & 
        (horizon_gdf['hzdept'] < o_end) & 
        (horizon_gdf['hzdepb'] > o_start)
    ]
    
    # Calculate total thickness of O horizon
    o_thickness = o_row['thickness']
    
    # If there are overlapping horizons, calculate adjusted thickness
    if not overlapping_horizons.empty:
        overlap_count = len(overlapping_horizons)
        # Calculate the adjusted thickness
        adjusted_thickness = o_thickness / overlap_count
    else:
        adjusted_thickness = o_thickness  # No overlaps means no adjustment
    
    # Store the result
    adjusted_thicknesses.append({
        'peiidref': o_row['peiidref'],
        'adjusted_thickness': adjusted_thickness
    })

# Step 5: Create a DataFrame from adjusted thicknesses
adjusted_thickness_df = pd.DataFrame(adjusted_thicknesses)

# Step 6: Sum adjusted thickness for each pedon ID
final_adjusted_thickness = adjusted_thickness_df.groupby('peiidref')['adjusted_thickness'].sum().reset_index()

# Rename the column for clarity
final_adjusted_thickness.rename(columns={'adjusted_thickness': 'total_adjusted_O_thickness'}, inplace=True)

# Display the result
print(final_adjusted_thickness)


       peiidref  total_adjusted_O_thickness
0        107546                         6.0
1        107547                         2.0
2        107548                         0.0
3        107549                         3.0
4        107550                        27.0
...         ...                         ...
23214   1679621                        12.0
23215   1679622                        39.0
23216   1679623                        10.0
23217   1679624                        12.0
23218   1679625                        60.0

[23219 rows x 2 columns]


Explanation of the Code:
Calculate Thickness: We calculate the thickness of each horizon as before.
Filter O Horizons: We filter for the horizons that contain "O".
Loop Through O Horizons:
For each O horizon, we check which other horizons overlap using the conditions for overlapping (the start of one horizon is less than the end of another and vice versa).
Count the number of overlapping horizons.
Adjust the thickness of the O horizon based on the count of overlaps.
Store Adjusted Thickness: We append the adjusted thicknesses to a list.
Create a DataFrame: Convert the list of adjusted thicknesses into a DataFrame.
Aggregate Results: Finally, group by peiidref and sum the adjusted thicknesses.
This method effectively handles the complexity of overlapping horizons and computes the desired adjusted sum for each unique pedon ID.

In [11]:
import geopandas as gpd
import pandas as pd

# Assuming you already have the horizon_gdf DataFrame loaded

# Step 1: Calculate the thickness of each horizon
horizon_gdf['thickness'] = horizon_gdf['hzdepb'] - horizon_gdf['hzdept']

# Step 2: Filter for horizons with "O" in the hzname
o_horizons = horizon_gdf[horizon_gdf['hzname'].str.contains('O', na=False)]

# Calculate the original sum of O thicknesses
original_thickness_sum = o_horizons.groupby('peiidref')['thickness'].sum().reset_index()
original_thickness_sum.rename(columns={'thickness': 'o_thickness_straight_sum'}, inplace=True)

# Step 3: Initialize a list to hold the adjusted thicknesses
adjusted_thicknesses = []

# Step 4: Loop through each O horizon and calculate adjusted thickness
for index, o_row in o_horizons.iterrows():
    o_start = o_row['hzdept']
    o_end = o_row['hzdepb']
    
    # Get other horizons that overlap with the current O horizon
    overlapping_horizons = horizon_gdf[
        (horizon_gdf['peiidref'] == o_row['peiidref']) & 
        (horizon_gdf['hzdept'] < o_end) & 
        (horizon_gdf['hzdepb'] > o_start)
    ]
    
    # Calculate total thickness of O horizon
    o_thickness = o_row['thickness']
    
    # If there are overlapping horizons, calculate adjusted thickness
    if not overlapping_horizons.empty:
        overlap_count = len(overlapping_horizons)
        # Calculate the adjusted thickness
        adjusted_thickness = o_thickness / overlap_count
    else:
        adjusted_thickness = o_thickness  # No overlaps means no adjustment
    
    # Store the result
    adjusted_thicknesses.append({
        'peiidref': o_row['peiidref'],
        'o_thickness_adjusted': adjusted_thickness
    })

# Step 5: Create a DataFrame from adjusted thicknesses
adjusted_thickness_df = pd.DataFrame(adjusted_thicknesses)

# Step 6: Sum adjusted thickness for each pedon ID
final_adjusted_thickness = adjusted_thickness_df.groupby('peiidref')['o_thickness_adjusted'].sum().reset_index()

# Step 7: Merge the original and adjusted thickness DataFrames
final_result = pd.merge(original_thickness_sum, final_adjusted_thickness, on='peiidref', how='outer')

# Display the final result
print(final_result)


       peiidref  o_thickness_straight_sum  o_thickness_adjusted
0        107546                       6.0                   6.0
1        107547                       2.0                   2.0
2        107548                       0.0                   0.0
3        107549                       3.0                   3.0
4        107550                      27.0                  27.0
...         ...                       ...                   ...
23214   1679621                      12.0                  12.0
23215   1679622                      39.0                  39.0
23216   1679623                      10.0                  10.0
23217   1679624                      12.0                  12.0
23218   1679625                      60.0                  60.0

[23219 rows x 3 columns]


In [12]:
# Filter for peiidref where the straight sum and adjusted sum are different
different_values = final_result[final_result['o_thickness_straight_sum'] != final_result['o_thickness_adjusted']]

# Select only the relevant columns
different_values = different_values[['peiidref', 'o_thickness_straight_sum', 'o_thickness_adjusted']]

# Display the results
print(different_values)

       peiidref  o_thickness_straight_sum  o_thickness_adjusted
11       107559                      52.0             32.000000
14       107563                      85.0             51.750000
21       107570                     100.0             69.500000
22       107571                      10.0              5.000000
23       107572                      19.0              7.833333
...         ...                       ...                   ...
22913   1670036                      26.0             13.000000
22983   1670250                      55.0             40.333333
23005   1671806                      10.0              5.000000
23019   1671821                      37.0             31.000000
23078   1675602                      16.0             14.000000

[172 rows x 3 columns]
